In [1]:
# Import the dependencies.
import json
import pandas as pd
import numpy as np
import urllib

import re

from sqlalchemy import create_engine
from urllib.parse import quote
import psycopg2

# from config import db_password
from config import g_key
import time

In [2]:
#  Add the clean movie function that takes in the argument, "movie".
def clean_movie(movie):
    movie = dict(movie)  # create a non-destructive copy
    alt_titles = {}
    # combine alternate titles into one list
    for key in [
        "Also known as",
        "Arabic",
        "Cantonese",
        "Chinese",
        "French",
        "Hangul",
        "hebrew",
        "Hepburn",
        "Japanese",
        "Literally",
        "Mandarin",
        "mcCune–reischauer",
        "Original title",
        "Polish",
        "Revised Romanization",
        "Romanized",
        "russian",
        "Simplified",
        "Traditional",
        "Yiddish",
    ]:
        if key in movie:
            alt_titles[key] = movie[key]
            movie.pop(key)
    if len(alt_titles) > 0:
        movie["alt_titles"] = alt_titles

    # merge column names
    def change_column_name(old_name, new_name):
        if old_name in movie:
            movie[new_name] = movie.pop(old_name)

    change_column_name("Adaptation by", "Writer(s)")
    change_column_name("Country of origin", "Country")
    change_column_name("Directed by", "Director")
    change_column_name("Distributed by", "Distributor")
    change_column_name("Edited by", "Editor(s)")
    change_column_name("Length", "Running time")
    change_column_name("Original release", "Release date")
    change_column_name("Music by", "Composer(s)")
    change_column_name("Produced by", "Producer(s)")
    change_column_name("Producer", "Producer(s)")
    change_column_name("Productioncompanies ", "Production company(s)")
    change_column_name("Productioncompany ", "Production company(s)")
    change_column_name("Released", "Release Date")
    change_column_name("Release Date", "Release date")
    change_column_name("Screen story by", "Writer(s)")
    change_column_name("Screenplay by", "Writer(s)")
    change_column_name("Story by", "Writer(s)")
    change_column_name("Theme music composer", "Composer(s)")
    change_column_name("Written by", "Writer(s)")

    return movie


In [3]:
# 1 Add the function that takes in three arguments;
# Wikipedia data, Kaggle metadata, and MovieLens rating data (from Kaggle)


def extract_transform_load(wiki_file, kaggle_file, ratings_file):
    # Read in the kaggle metadata and MovieLens ratings CSV files as Pandas DataFrames.
    kaggle_metadata = pd.read_csv(kaggle_file, low_memory=False)
    ratings = pd.read_csv(ratings_file, low_memory=False)

    # Open and read the Wikipedia data JSON file.
    with open(wiki_file, mode="r") as file:
        wiki_movies_raw = json.load(file)

    # Write a list comprehension to filter out TV shows.
    wiki_movies = [
        movie
        for movie in wiki_movies_raw
        if ("Director" in movie or "Directed by" in movie)
        and "imdb_link" in movie
        and "No. of episodes" not in movie
    ]

    # Write a list comprehension to iterate through the cleaned wiki movies list
    # and call the clean_movie function on each movie.
    clean_movies = [clean_movie(movie) for movie in wiki_movies]

    # Read in the cleaned movies list from Step 4 as a DataFrame.
    wiki_movies_df = pd.DataFrame(clean_movies)

    # Write a try-except block to catch errors while extracting the IMDb ID using a regular expression string and
    #  dropping any imdb_id duplicates. If there is an error, capture and print the exception.
    try:
        wiki_movies_df["imdb_id"] = wiki_movies_df["imdb_link"].str.extract(
            r"(tt\d{7})"
        )
        wiki_movies_df.drop_duplicates(subset="imdb_id", inplace=True)
    except:
        print("Something went wrong")

    #  Write a list comprehension to keep the columns that don't have null values from the wiki_movies_df DataFrame.
    wiki_columns_to_keep = [
        column
        for column in wiki_movies_df.columns
        if wiki_movies_df[column].isnull().sum() < len(wiki_movies_df) * 0.9
    ]
    wiki_movies_df = wiki_movies_df[wiki_columns_to_keep]

    # Create a variable that will hold the non-null values from the “Box office” column.
    box_office = wiki_movies_df["Box office"].dropna()

    # Convert the box office data created in Step 8 to string values using the lambda and join functions.
    box_office = box_office.apply(lambda x: " ".join(x) if type(x) == list else x)

    # Write a regular expression to match the six elements of "form_one" of the box office data.
    form_one = r"\$\s*\d+\.?\d*\s*[mb]illi?on"
    # Write a regular expression to match the three elements of "form_two" of the box office data.
    form_two = r"\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)"

    # Add the parse_dollars function.
    def parse_dollars(s):
        # if s is not a string, return NaN
        if type(s) != str:
            return np.nan

        # if input is of the form $###.# million
        if re.match(r"\$\s*\d+\.?\d*\s*milli?on", s, flags=re.IGNORECASE):

            # remove dollar sign and " million"
            s = re.sub("\$|\s|[a-zA-Z]", "", s)

            # convert to float and multiply by a million
            value = float(s) * 10 ** 6

            # return value
            return value

        # if input is of the form $###.# billion
        elif re.match(r"\$\s*\d+\.?\d*\s*billi?on", s, flags=re.IGNORECASE):

            # remove dollar sign and " billion"
            s = re.sub("\$|\s|[a-zA-Z]", "", s)

            # convert to float and multiply by a billion
            value = float(s) * 10 ** 9

            # return value
            return value

        # if input is of the form $###,###,###
        elif re.match(
            r"\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)", s, flags=re.IGNORECASE
        ):

            # remove dollar sign and commas
            s = re.sub("\$|,", "", s)

            # convert to float
            value = float(s)

            # return value
            return value

        # otherwise, return NaN
        else:
            return np.nan

    # Clean the box office column in the wiki_movies_df DataFrame.
    wiki_movies_df["box_office"] = box_office.str.extract(
        f"({form_one}|{form_two})", flags=re.IGNORECASE
    )[0].apply(parse_dollars)
    wiki_movies_df.drop("Box office", axis=1, inplace=True)

    # Clean the budget column in the wiki_movies_df DataFrame.
    budget = wiki_movies_df["Budget"].dropna()
    budget = budget.map(lambda x: " ".join(x) if type(x) == list else x)
    budget = budget.str.replace(r"\$.*[-—–](?![a-z])", "$", regex=True)
    budget = budget.str.replace(r"\[\d+\]\s*", "")
    wiki_movies_df["budget"] = budget.str.extract(
        f"({form_one}|{form_two})", flags=re.IGNORECASE
    )[0].apply(parse_dollars)

    # wiki_movies_df.drop('Budget', axis=1, inplace=True)
    wiki_movies_df.drop("Budget", axis=1, inplace=True)

    # Clean the release date column in the wiki_movies_df DataFrame.
    release_date = (
        wiki_movies_df["Release date"]
        .dropna()
        .apply(lambda x: " ".join(x) if type(x) == list else x)
    )
    date_form_one = r"(?:January|February|March|April|May|June|July|August|September|October|November|December)\s[123]\d,\s\d{4}"
    date_form_two = r"\d{4}.[01]\d.[123]\d"
    date_form_three = r"(?:January|February|March|April|May|June|July|August|September|October|November|December)\s\d{4}"
    date_form_four = r"\d{4}"
    wiki_movies_df["release_date"] = pd.to_datetime(
        release_date.str.extract(
            f"({date_form_one}|{date_form_two}|{date_form_three}|{date_form_four})"
        )[0],
        infer_datetime_format=True,
    )

    # Clean the running time column in the wiki_movies_df DataFrame.
    running_time = (
        wiki_movies_df["Running time"]
        .dropna()
        .apply(lambda x: " ".join(x) if type(x) == list else x)
    )
    running_time_extract = running_time.str.extract(
        r"(\d+)\s*ho?u?r?s?\s*(\d*)|(\d+)\s*m"
    )
    running_time_extract = running_time_extract.apply(
        lambda col: pd.to_numeric(col, errors="coerce")
    ).fillna(0)
    wiki_movies_df["running_time"] = running_time_extract.apply(
        lambda row: row[0] * 60 + row[1] if row[2] == 0 else row[2], axis=1
    )
    wiki_movies_df.drop("Running time", axis=1, inplace=True)

    # 2. Clean the Kaggle metadata.
    kaggle_metadata = kaggle_metadata[kaggle_metadata["adult"] == "False"].drop(
        "adult", axis="columns"
    )
    kaggle_metadata["video"] = kaggle_metadata["video"] == "True"
    kaggle_metadata["budget"] = kaggle_metadata["budget"].astype(int)
    kaggle_metadata["id"] = pd.to_numeric(kaggle_metadata["id"], errors="raise")
    kaggle_metadata["popularity"] = pd.to_numeric(
        kaggle_metadata["popularity"], errors="raise"
    )
    kaggle_metadata["release_date"] = pd.to_datetime(kaggle_metadata["release_date"])

    # 3. Merged the two DataFrames into the movies DataFrame.
    movies_df = pd.merge(
        wiki_movies_df, kaggle_metadata, on="imdb_id", suffixes=["_wiki", "_kaggle"]
    )

    # 4. Drop unnecessary columns from the merged DataFrame.
    movies_df.drop(
        columns=[
            "title_wiki",
            "release_date_wiki",
            "Language",
            "Production company(s)",
        ],
        inplace=True,
    )

    # 5. Add in the function to fill in the missing Kaggle data.
    def fill_missing_kaggle_data(df, kaggle_column, wiki_column):
        df[kaggle_column] = df.apply(
            lambda row: row[wiki_column]
            if row[kaggle_column] == 0
            else row[kaggle_column],
            axis=1,
        )
        df.drop(columns=wiki_column, inplace=True)

    # 6. Call the function in Step 5 with the DataFrame and columns as the arguments.
    fill_missing_kaggle_data(movies_df, "runtime", "running_time")
    fill_missing_kaggle_data(movies_df, "budget_kaggle", "budget_wiki")
    fill_missing_kaggle_data(movies_df, "revenue", "box_office")

    # 7. Filter the movies DataFrame for specific columns.
    movies_df = movies_df.loc[
        :,
        [
            "imdb_id",
            "id",
            "title_kaggle",
            "original_title",
            "tagline",
            "belongs_to_collection",
            "url",
            "imdb_link",
            "runtime",
            "budget_kaggle",
            "revenue",
            "release_date_kaggle",
            "popularity",
            "vote_average",
            "vote_count",
            "genres",
            "original_language",
            "overview",
            "spoken_languages",
            "Country",
            "production_companies",
            "production_countries",
            "Distributor",
            "Producer(s)",
            "Director",
            "Starring",
            "Cinematography",
            "Editor(s)",
            "Writer(s)",
            "Composer(s)",
            "Based on",
        ],
    ]

    # 8. Rename the columns in the movies DataFrame.
    movies_df.rename(
        {
            "id": "kaggle_id",
            "title_kaggle": "title",
            "url": "wikipedia_url",
            "budget_kaggle": "budget",
            "release_date_kaggle": "release_date",
            "Country": "country",
            "Distributor": "distributor",
            "Producer(s)": "producers",
            "Director": "director",
            "Starring": "starring",
            "Cinematography": "cinematography",
            "Editor(s)": "editors",
            "Writer(s)": "writers",
            "Composer(s)": "composers",
            "Based on": "based_on",
        },
        axis="columns",
        inplace=True,
    )

    # 9. Transform and merge the ratings DataFrame.
    rating_counts = (
        ratings.groupby(["movieId", "rating"], as_index=False)
        .count()
        .rename({"userId": "count"}, axis=1)
        .pivot(index="movieId", columns="rating", values="count")
    )
    rating_counts.columns = ["rating_" + str(col) for col in rating_counts.columns]
    movies_with_ratings_df = pd.merge(
        movies_df, rating_counts, left_on="kaggle_id", right_index=True, how="left"
    )
    movies_with_ratings_df[rating_counts.columns] = movies_with_ratings_df[
        rating_counts.columns
    ].fillna(0)

    # A long string that contains the necessary Postgres login information
    db_string = 'postgresql://postgres:%s@127.0.0.1:5432/movie_data'

    # This is all the information that SQLAlchemy needs to create a database engine.SQLAlchemy handles connections to different SQL databases and manages the conversion between data types. The way it handles all the communication and conversion is by creating a database engine.
    # Create the database engine with the following aka create the connection:
    engine = create_engine(db_string % urllib.parse.quote(g_key))

    # Add the movies_df DataFrame to a SQL database.
    with engine.connect() as conn, conn.begin():
        movies_df.to_sql(name="movies", con=conn, if_exists="replace")

    # Drop the ratings table in Postgres if needed.
    # Uncomment the line below to do so.
    # TODO: NEEDS WORK!
    # with engine.connect() as conn, conn.begin():
    #     RATINGS_DF.to_sql(name='ratings', con=engine, if_exists="replace")

    # This is better way to handle the connection.
    # Because it ensures that your connection is always closed, even if your program exits with an error. This is important to prevent data corruption.
    with engine.connect() as conn, conn.begin():
        rows_imported = 0
        # get the start_time from time.time()
        start_time = time.time()
        for data in pd.read_csv(f"{file_dir}/ratings.csv", chunksize=1000000):
            print(
                f"importing rows {rows_imported} to {rows_imported + len(data)}...",
                end="",
            )
            data.to_sql(name="ratings", con=conn, if_exists="append")
            rows_imported += len(data)

            # add elapsed time to final print out
            print(f"Done. {time.time() - start_time} total seconds elapsed")

    # I'm still returning the DFs to double check the data.
    return wiki_movies_df, movies_with_ratings_df, movies_df

In [4]:
# 10. Create the path to your file directory and variables for the three files.
file_dir = "./data"
# The Wikipedia data
wiki_file = f"{file_dir}/wikipedia-movies.json"
# The Kaggle metadata
kaggle_file = f"{file_dir}/movies_metadata.csv"
# The MovieLens rating data.
ratings_file = f"{file_dir}/ratings.csv"

In [5]:
# 11. Set the three variables equal to the function created in D1.
wiki_file, kaggle_file, ratings_file = extract_transform_load(
    wiki_file, kaggle_file, ratings_file
)

/var/folders/yp/_fjc7k152h37002p573ft1l00000gn/T/ipykernel_16093/3889997527.py:117: FutureWarning: The default value of regex will change from True to False in a future version.
  budget = budget.str.replace(r"\[\d+\]\s*", "")


importing rows 0 to 1000000...Done. 27.21491026878357 total seconds elapsed
importing rows 1000000 to 2000000...Done. 54.37783718109131 total seconds elapsed
importing rows 2000000 to 3000000...Done. 81.82245016098022 total seconds elapsed
importing rows 3000000 to 4000000...Done. 108.26476120948792 total seconds elapsed
importing rows 4000000 to 5000000...Done. 135.1173450946808 total seconds elapsed
importing rows 5000000 to 6000000...Done. 162.38547611236572 total seconds elapsed
importing rows 6000000 to 7000000...Done. 190.08617210388184 total seconds elapsed
importing rows 7000000 to 8000000...Done. 217.96546411514282 total seconds elapsed
importing rows 8000000 to 9000000...Done. 245.4246370792389 total seconds elapsed
importing rows 9000000 to 10000000...Done. 273.43496537208557 total seconds elapsed
importing rows 10000000 to 11000000...Done. 301.24231004714966 total seconds elapsed
importing rows 11000000 to 12000000...Done. 328.8115360736847 total seconds elapsed
importing r

In [6]:
# 12. Set the DataFrames from the return statement equal to the file names in Step 11.
wiki_movies_df = wiki_file
movies_with_ratings_df = kaggle_file
movies_df = ratings_file

In [ ]:
pdoptions.

In [32]:
# 13. Check the wiki_movies_df DataFrame. 
wiki_movies_df.head()

,url,year,imdb_link,title,Based on,...,imdb_id,box_office,budget,release_date,running_time
0,https://en.wikipedia.org/wiki/The_Adventures_o...,1990,https://www.imdb.com/title/tt0098987/,The Adventures of Ford Fairlane,"[Characters, by Rex Weiner]",...,tt0098987,21400000.0,20000000.0,1990-07-11,102.0
1,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",1990,https://www.imdb.com/title/tt0098994/,"After Dark, My Sweet","[the novel, After Dark, My Sweet, by, Jim Thom...",...,tt0098994,2700000.0,6000000.0,1990-05-17,114.0
2,https://en.wikipedia.org/wiki/Air_America_(film),1990,https://www.imdb.com/title/tt0099005/,Air America,"[Air America, by, Christopher Robbins]",...,tt0099005,57718089.0,35000000.0,1990-08-10,113.0
3,https://en.wikipedia.org/wiki/Alice_(1990_film),1990,https://www.imdb.com/title/tt0099012/,Alice,NaN,...,tt0099012,7331647.0,12000000.0,1990-12-25,106.0
4,https://en.wikipedia.org/wiki/Almost_an_Angel,1990,https://www.imdb.com/title/tt0099018/,Almost an Angel,NaN,...,tt0099018,6939946.0,25000000.0,1990-12-19,95.0


In [9]:
# 14. Check the movies_with_ratings_df DataFrame.
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 10)
pd.set_option('display.width', 1000)
movies_with_ratings_df.head()

,imdb_id,kaggle_id,title,original_title,tagline,...,rating_3.0,rating_3.5,rating_4.0,rating_4.5,rating_5.0
0,tt0098987,9548,The Adventures of Ford Fairlane,The Adventures of Ford Fairlane,Kojak. Columbo. Dirty Harry. Wimps.,...,0.0,0.0,0.0,0.0,0.0
1,tt0098994,25501,"After Dark, My Sweet","After Dark, My Sweet",All they risked was everything.,...,0.0,0.0,0.0,0.0,0.0
2,tt0099005,11856,Air America,Air America,The few. The proud. The totally insane.,...,0.0,0.0,0.0,0.0,0.0
3,tt0099012,8217,Alice,Alice,NaN,...,0.0,0.0,0.0,0.0,0.0
4,tt0099018,25943,Almost an Angel,Almost an Angel,Who does he think he is?,...,26.0,37.0,46.0,16.0,11.0


In [10]:
# 15. Check the movies_df DataFrame. 
movies_df.head()

,imdb_id,kaggle_id,title,original_title,tagline,...,cinematography,editors,writers,composers,based_on
0,tt0098987,9548,The Adventures of Ford Fairlane,The Adventures of Ford Fairlane,Kojak. Columbo. Dirty Harry. Wimps.,...,Oliver Wood,Michael Tronick,"[David Arnott, James Cappe]","[Cliff Eidelman, Yello]","[Characters, by Rex Weiner]"
1,tt0098994,25501,"After Dark, My Sweet","After Dark, My Sweet",All they risked was everything.,...,Mark Plummer,Howard E. Smith,"[James Foley, Robert Redlin]",Maurice Jarre,"[the novel, After Dark, My Sweet, by, Jim Thom..."
2,tt0099005,11856,Air America,Air America,The few. The proud. The totally insane.,...,Roger Deakins,"[John Bloom, Lois Freeman-Fox]","[John Eskow, Richard Rush]",Charles Gross,"[Air America, by, Christopher Robbins]"
3,tt0099012,8217,Alice,Alice,NaN,...,Carlo Di Palma,Susan E. Morse,Woody Allen,NaN,NaN
4,tt0099018,25943,Almost an Angel,Almost an Angel,Who does he think he is?,...,Russell Boyd,David Stiven,Paul Hogan,Maurice Jarre,NaN


In [11]:
# A long string that contains the necessary Postgres login information
db_string = 'postgresql://postgres:%s@127.0.0.1:5432/movie_data'

# This is all the information that SQLAlchemy needs to create a database engine.SQLAlchemy handles connections to different SQL databases and manages the conversion between data types. The way it handles all the communication and conversion is by creating a database engine.
# Create the database engine with the following aka create the connection:
engine = create_engine(db_string % urllib.parse.quote(g_key))
# Pandas Connections to Postgres to check if the import worked
with engine.connect() as conn, conn.begin():
    movie_data_from_db = pd.read_sql("select count(*) from movies", con=conn)
    print(movie_data_from_db)

   count
0   6052


In [12]:
# A long string that contains the necessary Postgres login information
db_string = 'postgresql://postgres:%s@127.0.0.1:5432/movie_data'

# This is all the information that SQLAlchemy needs to create a database engine.SQLAlchemy handles connections to different SQL databases and manages the conversion between data types. The way it handles all the communication and conversion is by creating a database engine.
# Create the database engine with the following aka create the connection:
engine = create_engine(db_string % urllib.parse.quote(g_key))
# Pandas Connections to Postgres to check if the import worked
with engine.connect() as conn, conn.begin():
    rating_data_from_db = pd.read_sql("select count(*) from ratings", con=conn)
    print(rating_data_from_db)

      count
0  52048578
